In [4]:
# ==============================================================================
# BƯỚC 1: CÀI ĐẶT CÁC THƯ VIỆN CẦN THIẾT
# ==============================================================================
# Chạy dòng lệnh này trong terminal hoặc cell của Jupyter/Colab
!pip3 install networkx matplotlib scikit-learn python-louvain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 26.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 26.5 MB/s eta 0:00:00
  Created wheel for python-louvain: filename=python_louvain-0.16-py3-none-any.whl size=9460 sha256=5171685398ca15fca6d6f6a35051b8d50d9dd4a99973c270bd8f2ca425ea6c27
  Stored in directory: /Users/taminh/Library/Caches/pip/wheels/40/f1/e3/485b698c520fa0baee1d07897abc7b8d6479b7d199ce96f4af
Successfully built python-louvain

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [1]:

# ==============================================================================
# STEP 2: IMPORT LIBRARIES
# ==============================================================================
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from community import community_louvain
from sklearn.cluster import SpectralClustering
from sklearn.metrics.cluster import adjusted_rand_score

# ==============================================================================
# EXERCISE 2: Compare 3 algorithms (Louvain, Girvan-Newman, Spectral)
# ==============================================================================
print("--- EXERCISE 2: Comparing algorithms ---")

# Use the classic "Zachary's Karate Club" graph
G_karate = nx.karate_club_graph()

# 1. Louvain Algorithm
partition_louvain = community_louvain.best_partition(G_karate)
num_louvain_communities = len(set(partition_louvain.values()))
print(f"[Louvain]        => Found {num_louvain_communities} communities.")

# 2. Girvan-Newman Algorithm
# This algorithm returns a sequence of partitions, we take the first one
comp_gn = nx.community.girvan_newman(G_karate)
partition_gn = tuple(sorted(c) for c in next(comp_gn))
num_gn_communities = len(partition_gn)
print(f"[Girvan-Newman]  => Found {num_gn_communities} communities in the first split.")

# 3. Spectral Clustering
# This algorithm requires us to specify the number of clusters (k) beforehand
k = 2  # Let's assume we want to find 2 clusters (based on the original story of the Karate graph)
adj_matrix = nx.to_numpy_array(G_karate)
sc = SpectralClustering(n_clusters=k, affinity='precomputed', assign_labels='kmeans', random_state=42)
partition_spectral = sc.fit_predict(adj_matrix)
print(f"[Spectral]       => Found {k} communities as requested.")
print("-" * 50 + "\n")


# ==============================================================================
# EXERCISE 3: Simulate changing communities and track Modularity
# ==============================================================================
print("--- EXERCISE 3: Simulating changing communities ---")

# Create a graph with a clear community structure (T=1)
sizes = [25, 25, 25]  # 3 communities, each with 25 nodes
# Connection probability: high inside (0.7), low outside (0.02)
probs = [[0.7, 0.02, 0.02], [0.02, 0.7, 0.02], [0.02, 0.02, 0.7]]
G1 = nx.stochastic_block_model(sizes, probs, seed=42)

# Define the ground truth communities
ground_truth_communities = [set(range(0, 25)), set(range(25, 50)), set(range(50, 75))]

# Calculate Modularity at T=1
modularity_t1 = nx.community.modularity(G1, ground_truth_communities)
print(f"Modularity at T=1 (clear structure): {modularity_t1:.4f}")

# "Blur" community boundaries by adding random "bridge" edges (T=2)
G2 = G1.copy()
num_bridge_edges = 30
for _ in range(num_bridge_edges):
    # Randomly select 2 nodes from 2 different communities
    node1 = np.random.choice(list(ground_truth_communities[0]))
    node2 = np.random.choice(list(ground_truth_communities[1]))
    if not G2.has_edge(node1, node2):
        G2.add_edge(node1, node2)

# Recalculate Modularity at T=2 on G2's structure, but still based on the original community partition
modularity_t2 = nx.community.modularity(G2, ground_truth_communities)
print(f"Modularity at T=2 (blurrier structure):  {modularity_t2:.4f}")
print("-" * 50 + "\n")


# ==============================================================================
# EXERCISE 4: Evaluate accuracy using LFR data
# ==============================================================================
print("--- EXERCISE 4: Evaluating Louvain's accuracy ---")

# Experiment with 2 different levels of community "blurriness"
mixing_params = [0.1, 0.6] # mu=0.1 (clear), mu=0.6 (blurry)

for mu in mixing_params:
    # 1. Create LFR benchmark graph
    # The LFR graph is specially designed to have a known community structure
    G_lfr = nx.LFR_benchmark_graph(n=250, tau1=3, tau2=1.5, mu=mu, average_degree=5,
                                   min_community=20, seed=42)

    # Extract the ground truth community labels from the graph
    ground_truth_comms_lfr = {frozenset(G_lfr.nodes[v]['community']) for v in G_lfr}
    ground_truth_labels = [0] * len(G_lfr.nodes)
    for i, comm in enumerate(ground_truth_comms_lfr):
        for node in comm:
            ground_truth_labels[node] = i

    # 2. Run the Louvain algorithm to find communities
    detected_partition = community_louvain.best_partition(G_lfr)
    detected_labels = list(detected_partition.values())

    # 3. Evaluate accuracy using the Adjusted Rand Index (ARI)
    # ARI = 1.0 is a perfect match, ARI = 0.0 is a random guess
    ari_score = adjusted_rand_score(ground_truth_labels, detected_labels)
    
    print(f"Results with mixing parameter (mu) = {mu}:")
    print(f"=> Louvain Accuracy (ARI Score): {ari_score:.4f}")

print("-" * 50)

--- EXERCISE 2: Comparing algorithms ---
[Louvain]        => Found 4 communities.
[Girvan-Newman]  => Found 2 communities in the first split.
[Spectral]       => Found 2 communities as requested.
--------------------------------------------------

--- EXERCISE 3: Simulating changing communities ---
Modularity at T=1 (clear structure): 0.6074
Modularity at T=2 (blurrier structure):  0.5695
--------------------------------------------------

--- EXERCISE 4: Evaluating Louvain's accuracy ---
Results with mixing parameter (mu) = 0.1:
=> Louvain Accuracy (ARI Score): 0.9319
Results with mixing parameter (mu) = 0.6:
=> Louvain Accuracy (ARI Score): 0.0158
--------------------------------------------------
